# Homework 3
## Joshua Chen


### Problem 2 

We are interested in solving the following low rank matrix problem. Given a sparse observation pattern $G =(g_{ij})_{1\le i,j\le n}\in \{0,1\}^{n\times n}$ and a data matrix $A=(a_{ij})_{1 \le i, j \le n} \in \mathbb{R}^{n \times n}$, our goal is to recover a low rank matrix pair, $B \in \mathbb{R}^{n\times r}, C \in \mathbb{R}^{r \times n}$ by minimizing

$\displaystyle \min_{B,C} \sum_{i=1}^n\sum_{j=1}^n g_{ij}(a_{ij} - e_i^T B(Ce_j))^2 + \frac{\mu}{2}(\|B\|_{\mathcal{F}}^2 + \|C\|_{\mathcal{F}}^2)$, where $\mu$ is a small constant, making the optimization problem non-degenerate. (i.e. regularization constant... this is standard in all least squares types optimization problems...)

Recognize that we can write this entirely in terms of the Frobenius inner product $\left( \cdot ,\cdot\right)_{\mathcal{F}}$:

$\displaystyle \min_{B,C} \left( G, (A -BC)\circ(A-BC)\right)_{\mathcal{F}} + \frac{\mu}{2} \left( B, B \right)_{\mathcal{F}} + \frac{\mu}{2} \left( C,C\right)_{\mathcal{F}}$.

where $\circ$ the Hadamard product has already been defined in Homework 1.

The gradient is:

$\nabla f = \left[\begin{array}{cc} \nabla_Bf ,\nabla_Cf\end{array} \right]$, where $\nabla_Bf \in \mathbb{R}^{n\times r}$, $\nabla_Cf \in \mathbb{R}^{r\times n}$,

where 

$\displaystyle\nabla_B f = -2 \big(G\circ(A - BC)\big)C^T + \mu B$ and $\displaystyle\nabla_C f = -2B^T\big(G \circ(A-BC) \big) + \mu C$.


We also need to Hessian for Newton type methods. Below I test Newton and Trust-Region Newton. Both require the Hessian and the Gradient.

The Hessian, in block form, is:
$H = \left[\begin{array}{cc} H_{BB} & H_{BC} \\ H_{CB} & H_{CC} \end{array}\right] $

where

$[H_{BB}]_{lmpq} = \frac{\partial}{\partial b_{pq}} [\nabla_B f]_{lm} = 2 \sum\limits_{j=1}^n g_{lj}\delta_{lp}c_{qj}c_{mj} + \mu \delta_{lp}\delta_{mq}$

$[H_{BC}]_{lmpq} = \frac{\partial}{\partial c_{qp}} [\nabla_B f]_{lm} = 2g_{lp}b_{lq}c_{mp} -2(G\circ (A-BC))_{lp} \delta_{mq}$

$[H_{CB}]_{lmpq} = \frac{\partial}{\partial b_{pq}} [\nabla_C f]_{lm} = 2g_{pl}b_{pm}c_{ql} -2(G\circ (A-BC))_{pl} \delta_{mq}$

$[H_{CC}]_{lmpq} = \frac{\partial}{\partial c_{qp}} [\nabla_C f]_{lm} = 2\sum\limits_{i=1}^n b_{im}g_{il} b_{iq}\delta_{lp} + \mu \delta_{mq}\delta_{lp}$

(This was derived by Tom O'leary Roseberry and Josh Chen together on the whiteboard.)

In [1]:
import numpy as np
import numpy.linalg as la

class Problem2:
    def __init__(self,A,B,C,G,mu=1e-2):
        self.A = A
        self.B = B
        self.C = C
        self.G = G
        self.mu =mu
        self.n,self.r = self.B.shape
        
    def cost(self,B=None,C=None):
        if B is None:
            B = self.B
        if C is None:
            C = self.C
        A_minus_BC = self.A - np.matmul(B,C)
        A_minus_BCsquared = np.multiply(A_minus_BC,A_minus_BC)
        misfit = np.sum(np.multiply(self.G,A_minus_BCsquared))
        reg_B = 0.5*self.mu*np.sum(np.multiply(B,B))
        reg_C = 0.5*self.mu*np.sum(np.multiply(C,C))
        return [misfit,reg_B, reg_C,misfit+reg_B+reg_C]
    
    def gradient(self): #gradient as two matrices
        reggrad_B = self.mu*self.B 
        reggrad_C = self.mu*self.C

        A_minus_BC = self.A - np.matmul(self.B,self.C)
        G_circ_A_minus_BC = np.multiply(self.G,A_minus_BC)
        
        costgrad_B = -2.*np.matmul(G_circ_A_minus_BC,self.C.T)
        costgrad_C = -2.*np.matmul(self.B.T,G_circ_A_minus_BC)
        
        grad_B = costgrad_B + reggrad_B
        grad_C = costgrad_C + reggrad_C
        return grad_B, grad_C
    
    def gradientnorm(self):
        gradient = self.gradient()
        return np.sqrt(la.norm(gradient[0])**2.0+la.norm(gradient[1])**2.0)
    
    def partialgradientnorm(self,minimizeinB):
        gradient = self.gradient()
        if minimizeinB:
            return la.norm(gradient[0])
        else:
            return la.norm(gradient[1])
        
    def TakeAlternatingStep(self,MinimizeInB,alpha,grad_B,grad_C):
        if MinimizeInB:
            self.B -= alpha*grad_B
        else:
            self.C -= alpha*grad_C
            
    def TakeStep(self,alpha,p_B,p_C):
        self.B +=p_B
        self.C +=p_C
    
    def HessianMatrixProduct(self, X = None, Y=None):
        if X is None:
            X = np.zeros_like(self.B)
        if Y is None:
            Y = np.zeros_like(self.C)
        if X.shape != self.B.shape or Y.shape != self.C.shape:
            raise  
        XC = np.matmul(X,self.C)
        G_circ_XC = np.multiply(self.G,XC) 
        H_BB_X = np.matmul(G_circ_XC,self.C.T) + self.mu*X
        A_minus_BC = self.A - np.matmul(self.B,self.C)
        BY = np.matmul(self.B,Y)
        G_circ_BY = np.multiply(self.G,BY)
        G_circ_A_minus_BC = np.multiply(self.G,A_minus_BC)
        H_BC_Y = 2*np.matmul(self.C,G_circ_BY.T) - 2*np.matmul(Y,G_circ_A_minus_BC.T)
        H_CB_X = 2*np.matmul(G_circ_XC.T,self.B) - 2*np.matmul(G_circ_A_minus_BC.T,X)
        H_CC_Y = np.matmul(self.B.T,G_circ_BY) + self.mu*Y
        return [H_BB_X+H_BC_Y.T,H_CB_X+ H_CC_Y.T]
    
    def HessianFrobeniusInner(self, X = None,Y=None):
        if X is None:
            X = np.zeros_like(self.B)
        if Y is None:
            Y = np.zeros_like(self.C)
        H_XY = self.HessianMatrixProduct(X,Y)
        return np.sum(np.multiply(X,H_XY[0])) + np.sum(np.multiply(Y.T,H_XY[1]))

    def Hessian(self): #tensor Hessian
        n,r = self.B.shape
        dim = n*r
        H = np.zeros((2*n,r,2*n,r))
        A_BC = self.A - np.matmul(self.B,self.C)
        G_had_A_BC = np.multiply(self.G,A_BC)
        for l in range(n):
            for m in range(r):
                for p in range(n):
                    for q in range(r):
                        #HBB
                        H[l,m,p,q] += self.mu*(l==p)*(m==q)
                        #HBC
                        H[n+l,m,p,q] += 2*self.G[l,p]*self.B[l,q]*self.C[m,p]\
                            -2*G_had_A_BC[l,p]*(m == q)
                        #HCB
                        H[l,m,n+p,q] += 2*self.G[p,l]*self.B[p,m]*self.C[q,l]\
                            - 2*G_had_A_BC[p,l]*(m == q)
                        #HCC
                        H[n+l,m,n+p,q] += self.mu*(l==p)*(m==q)
                        for j in range(n):
#                             HBB
                            H[l,m,p,q] += 2*self.G[l,j]*(l==p)\
                            *self.C[q,j]*self.C[m,j]
#                             HCC
                            H[n+l,m,n+p,q] += 2*self.G[j,l]*(l==p)\
                            *self.B[j,m]*self.B[j,q]
        return H
        
    def Gradient(self): #gradient as a 2*n x r matrix
        n,r = self.B.shape
        gradient = np.zeros((2*n,r))
        A_BC = self.A - np.matmul(self.B,self.C)
        G_had_A_BC = np.multiply(self.G,A_BC)
        for l in range(n):
            for m in range(r):
                #G_B
                gradient[l,m] += self.mu*self.B[l,m]
                #G_C
                gradient[n+l,m] += self.mu*self.C[m,l]
                for j in range(n):
                    #G_B
                    gradient[l,m] += -2*G_had_A_BC[l,j]*self.C[m,j]
                    #G_C
                    gradient[n+l,m] += -2*G_had_A_BC[j,l]*self.B[j,m]
        return gradient
    
    def SplitAndTranspose(self,p):
        p_B,p_CT = np.split(p,2)
        return p_B, p_CT.T
    
    def NewtonDirection(self):
        g = self.Gradient()
        H = self.Hessian()
        p = np.linalg.tensorsolve(H,-g)
        return self.SplitAndTranspose(p)
    
    def PredictedReduction(self,p):
        g = self.gradient()
        gp = np.sum(np.multiply(g[0],p[0])) + np.sum(np.multiply(g[1],p[1]))
        pHp = self.HessianFrobeniusInner(X = p[0],Y=p[1])
        return -gp -0.5*pHp
    
    def FrobeniusNorm(self,p):
        p_norm_2 = np.sum(np.multiply(p[0],p[0])) +np.sum( np.multiply(p[1],p[1])) 
        return np.sqrt(p_norm_2)
        
    def RescaleDirection(self,p,alpha):
        return [alpha*p[0],alpha*p[1]]
        
        
                
                
        
        
    

Now I need to generate the data matrix $A$, sparse observational pattern $G$ and initialize the control variables $B$ and $C$


Because it doesn't really matter I will take $A \sim \mathcal{N}(0,\sigma^2)^{n \times n}$

I take $B$ and $C$ to be a pair of low rank recovery of identity (i.e. they are both identity padded with extra zeros in the strictly rectuangular rows or columns see $\texttt{numpy.matlib.eye}$ for implementation). And $B*C = I_r \in \mathbb{R}^{n\times n}$, identity up to the $r^{th}$ diagonal and then all zeros.

In [126]:
import scipy.optimize as spopt
import numpy as np
import numpy.linalg as la

def AlternatingMinimization(problem_handler,outer_tolerance=1e-6,inner_tolerance = 1e-4,max_iters =10000,back_track = False,
                     c_armijo = 1e-5,rho_armijo =0.9,verbose = False,min_length = 1e-8):
    
    problem = problem_handler

    grad_norm0 = problem.gradientnorm()
    grad_norm = grad_norm0
    
    MinimizeInB = True #start with minimizing in B...
    print("Now optimizing over B")
    partialgrad_norm0 = problem.partialgradientnorm(MinimizeInB)
    partialgrad_norm = partialgrad_norm0
    
    print("\n{0:5} {1:9} {2:9} {3:9} {4:9} {5:9} {6:11} {7:11} {8:11}".format(
                          "Its",  "cost", "misfit","reg_B","reg_C","||grad||","||grad_B||","||grad_C||","alpha" ))
    if verbose:
        misfit,regB,regC,cost = problem.cost()
        partialgradB_norm = problem.partialgradientnorm(True)
        partialgradC_norm = problem.partialgradientnorm(False)
        print("\n{0:<5s} {1:<9.1e} {2:<9.1e} {3:<9.1e} {4:<9.1e} {5:<9.1e} {6:<11.1e} {7:<11.1e} {8:<11s}".format(
                  "-",  cost, misfit,regB,regC,grad_norm,partialgradB_norm,partialgradC_norm, "-"))
            
    outeriters = 0
    while(grad_norm > outer_tolerance*grad_norm0  and outeriters <= 10000):
        
        inneriters = 0
        while partialgrad_norm > inner_tolerance*partialgrad_norm0 and inneriters <= max_iters:
            grad_B, grad_C = problem.gradient()
            
            if back_track:
                misfit, regB, regC, cost = problem.cost()
                cost_old = cost
                alpha = 1.0
                while alpha > min_length and cost > cost_old - c_armijo*alpha*partialgrad_norm:
                    if MinimizeInB:
                        B_tilde = problem.B - alpha*grad_B
                        misfit,regB,regC,cost = problem.cost(B = B_tilde)
                    else:
                        C_tilde = problem.C - alpha*grad_C
                        misfit,regB,regC,cost = problem.cost(C = C_tilde)
                    alpha *= rho_armijo
                problem.TakeAlternatingStep(MinimizeInB,alpha,grad_B,grad_C)
                
            partialgrad_norm = problem.partialgradientnorm(MinimizeInB)
            grad_norm = problem.gradientnorm()
            inneriters += 1
        
        if verbose:
            partialgradB_norm = problem.partialgradientnorm(True)
            partialgradC_norm = problem.partialgradientnorm(False)
            misfit, regB, regC, cost = problem.cost()
            print("\n{0:<5d} {1:<9.1e} {2:<9.1e} {3:<9.1e} {4:<9.1e} {5:<9.1e} {6:<11.1e} {7:<11.1e} {8:<11.1e}".format(
                  inneriters,  cost, misfit,regB,regC,grad_norm,partialgradB_norm,partialgradC_norm, alpha))
        
        MinimizeInB = not MinimizeInB
        if MinimizeInB:
            print("Now optimizing over B")
            partialgrad_norm = problem.partialgradientnorm(True)
        else:
            print("Now optimizing over C")
            partialgrad_norm = problem.partialgradientnorm(False)
        partialgrad_norm0 = partialgrad_norm
        outeriters +=1
        
        #adhoc, just want the inner tolerance to be relaxed after each outer iteration
        inner_tolerance = inner_tolerance**0.99
    return problem.B,problem.C 

Alternating Gradient Descent Minimization

In [127]:
#test n = 100, r = 10
import numpy as np
import numpy.linalg as la

n = 100
r = 10
B =  np.random.rand(n,r)
C =  np.random.rand(r,n)
A = np.matmul(B,C) #generate a rank-r matrix A. Otherwise, this problem is pointless. 
#Needs to be rank-r for this problem to be meaningful.

mean = 0.0
sigma = 5.0
B = np.random.normal(loc=mean,scale=sigma,size=(n,r))
C = np.random.normal(loc=mean,scale=sigma,size=(r,n))

#Not too sparse, but whatever, TA doesnt care how sparse
G = np.random.randint(2, size=(n,n))

problem = Problem2(A,B,C,G)


B,C = AlternatingMinimization(problem,back_track = True,verbose=True)

print("Final Frobenius distance:"+str(np.linalg.norm(A-np.matmul(B,C))))

Now optimizing over B

Its   cost      misfit    reg_B     reg_C     ||grad||  ||grad_B||  ||grad_C||  alpha      

-     3.1e+07   3.1e+07   1.3e+02   1.2e+02   6.2e+05   4.3e+05     4.4e+05     -          

37    2.6e+04   2.6e+04   3.7e-02   1.2e+02   4.1e+02   4.0e+01     4.1e+02     3.0e-04    
Now optimizing over C

193   1.7e+03   1.6e+03   3.7e-02   1.2e+02   7.2e+03   7.2e+03     4.5e-02     3.5e-01    
Now optimizing over B

1794  3.6e+02   2.4e+02   2.8e-02   1.2e+02   1.4e+01   8.6e-01     1.4e+01     6.9e-05    
Now optimizing over C

588   2.4e+02   1.3e+02   2.8e-02   1.1e+02   4.8e+02   4.8e+02     1.8e-03     3.5e-01    
Now optimizing over B

2814  1.7e+02   6.2e+01   3.1e-02   1.1e+02   4.7e+00   6.9e-02     4.7e+00     6.9e-05    
Now optimizing over C

445   1.4e+02   3.2e+01   3.1e-02   1.1e+02   3.2e+02   3.2e+02     7.3e-04     3.5e-01    
Now optimizing over B

2165  1.2e+02   1.0e+01   3.3e-02   1.1e+02   3.0e+00   5.5e-02     3.0e+00     7.6e-05    
Now optim


279   7.5e+01   3.4e-02   6.3e-02   7.5e+01   1.5e+00   7.6e-01     1.2e+00     1.0e-04    
Now optimizing over C

22    7.5e+01   2.6e-01   6.3e-02   7.4e+01   7.3e+01   7.3e+01     1.3e-02     2.8e-01    
Now optimizing over B

259   7.4e+01   3.3e-02   6.4e-02   7.4e+01   1.5e+00   8.3e-01     1.2e+00     1.0e-04    
Now optimizing over C

22    7.4e+01   2.5e-01   6.4e-02   7.4e+01   7.2e+01   7.2e+01     1.3e-02     2.8e-01    
Now optimizing over B

243   7.4e+01   3.3e-02   6.5e-02   7.4e+01   1.5e+00   9.0e-01     1.2e+00     1.0e-04    
Now optimizing over C

21    7.4e+01   2.5e-01   6.5e-02   7.3e+01   7.2e+01   7.2e+01     1.5e-02     2.8e-01    
Now optimizing over B

230   7.4e+01   3.2e-02   6.5e-02   7.3e+01   1.6e+00   9.6e-01     1.2e+00     1.0e-04    
Now optimizing over C

21    7.3e+01   2.4e-01   6.5e-02   7.3e+01   7.1e+01   7.1e+01     1.6e-02     2.8e-01    
Now optimizing over B

214   7.3e+01   3.2e-02   6.6e-02   7.3e+01   1.6e+00   1.1e+00     1.2e+00    


8     6.3e+01   4.5e-02   7.8e-02   6.3e+01   5.9e+00   5.8e+00     1.1e+00     1.2e-04    
Now optimizing over C

8     6.3e+01   1.6e-01   7.8e-02   6.3e+01   5.3e+01   5.3e+01     9.7e-02     2.3e-01    
Now optimizing over B

7     6.3e+01   4.5e-02   7.8e-02   6.3e+01   6.3e+00   6.2e+00     1.2e+00     1.2e-04    
Now optimizing over C

8     6.3e+01   1.6e-01   7.8e-02   6.3e+01   5.2e+01   5.2e+01     1.0e-01     2.3e-01    
Now optimizing over B

7     6.3e+01   4.5e-02   7.9e-02   6.3e+01   6.2e+00   6.1e+00     1.2e+00     1.2e-04    
Now optimizing over C

7     6.3e+01   1.7e-01   7.9e-02   6.2e+01   5.4e+01   5.4e+01     1.3e-01     2.3e-01    
Now optimizing over B

7     6.2e+01   4.2e-02   7.9e-02   6.2e+01   6.3e+00   6.2e+00     1.2e+00     1.2e-04    
Now optimizing over C

7     6.2e+01   1.7e-01   7.9e-02   6.2e+01   5.6e+01   5.6e+01     1.5e-01     2.5e-01    
Now optimizing over B

7     6.2e+01   4.2e-02   7.9e-02   6.2e+01   5.9e+00   5.8e+00     1.1e+00    


4     5.5e+01   4.6e-02   8.3e-02   5.5e+01   1.2e+01   1.2e+01     9.0e-01     1.4e-04    
Now optimizing over C

4     5.5e+01   1.1e-01   8.3e-02   5.5e+01   3.8e+01   3.8e+01     2.5e-01     2.3e-01    
Now optimizing over B

4     5.5e+01   4.7e-02   8.4e-02   5.5e+01   1.2e+01   1.2e+01     9.3e-01     1.3e-04    
Now optimizing over C

4     5.5e+01   1.1e-01   8.4e-02   5.5e+01   3.8e+01   3.8e+01     2.8e-01     2.3e-01    
Now optimizing over B

4     5.5e+01   4.9e-02   8.4e-02   5.5e+01   1.3e+01   1.3e+01     9.4e-01     1.3e-04    
Now optimizing over C

4     5.5e+01   1.1e-01   8.4e-02   5.5e+01   3.8e+01   3.8e+01     3.5e-01     2.3e-01    
Now optimizing over B

3     5.5e+01   5.2e-02   8.4e-02   5.5e+01   1.3e+01   1.3e+01     1.2e+00     1.3e-04    
Now optimizing over C

4     5.5e+01   1.1e-01   8.4e-02   5.4e+01   3.6e+01   3.6e+01     3.9e-01     2.1e-01    
Now optimizing over B

4     5.5e+01   5.2e-02   8.4e-02   5.4e+01   1.0e+01   1.0e+01     9.4e-01    


2     4.9e+01   5.8e-02   8.8e-02   4.9e+01   1.8e+01   1.8e+01     8.2e-01     1.6e-04    
Now optimizing over C

2     4.9e+01   8.9e-02   8.8e-02   4.9e+01   3.0e+01   3.0e+01     4.7e-01     2.1e-01    
Now optimizing over B

2     4.9e+01   5.9e-02   8.8e-02   4.9e+01   1.8e+01   1.7e+01     8.4e-01     1.6e-04    
Now optimizing over C

2     4.9e+01   8.9e-02   8.8e-02   4.9e+01   2.9e+01   2.9e+01     5.0e-01     2.1e-01    
Now optimizing over B

2     4.9e+01   6.0e-02   8.8e-02   4.9e+01   1.7e+01   1.7e+01     8.5e-01     1.6e-04    
Now optimizing over C

2     4.9e+01   9.0e-02   8.8e-02   4.9e+01   2.9e+01   2.9e+01     5.2e-01     2.1e-01    
Now optimizing over B

2     4.9e+01   6.1e-02   8.9e-02   4.9e+01   1.7e+01   1.7e+01     8.7e-01     1.6e-04    
Now optimizing over C

2     4.9e+01   9.0e-02   8.9e-02   4.9e+01   2.9e+01   2.9e+01     5.4e-01     2.1e-01    
Now optimizing over B

2     4.9e+01   6.1e-02   8.9e-02   4.9e+01   1.7e+01   1.7e+01     8.8e-01    

...

1     4.4e+00   1.6e-03   8.4e-01   3.6e+00   9.1e-01   7.9e-01     4.3e-01     9.7e-03    
Now optimizing over B

1     4.4e+00   1.1e-03   8.4e-01   3.6e+00   7.7e-01   6.1e-01     4.7e-01     3.4e-03    
Now optimizing over C

1     4.4e+00   1.6e-03   8.4e-01   3.6e+00   8.6e-01   7.5e-01     4.2e-01     1.1e-02    
Now optimizing over B

1     4.4e+00   1.1e-03   8.4e-01   3.6e+00   8.2e-01   6.5e-01     5.0e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.8e-03   8.4e-01   3.6e+00   9.8e-01   8.5e-01     4.8e-01     1.1e-02    
Now optimizing over B

1     4.4e+00   1.1e-03   8.4e-01   3.6e+00   8.2e-01   6.4e-01     5.1e-01     3.4e-03    
Now optimizing over C

1     4.4e+00   1.7e-03   8.4e-01   3.6e+00   8.9e-01   7.7e-01     4.5e-01     1.1e-02    
Now optimizing over B

1     4.4e+00   1.2e-03   8.4e-01   3.6e+00   8.4e-01   6.5e-01     5.3e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.6e-03   8.4e-01   3.6e+00   8.9e-01   7.9e-01     4.2e-01     


1     4.4e+00   1.0e-03   8.5e-01   3.6e+00   7.6e-01   5.7e-01     5.1e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.5e-03   8.5e-01   3.6e+00   8.3e-01   7.2e-01     4.1e-01     9.7e-03    
Now optimizing over B

1     4.4e+00   9.1e-04   8.5e-01   3.6e+00   6.8e-01   5.2e-01     4.5e-01     3.4e-03    
Now optimizing over C

1     4.4e+00   1.5e-03   8.5e-01   3.6e+00   7.9e-01   6.7e-01     4.0e-01     1.1e-02    
Now optimizing over B

1     4.4e+00   9.6e-04   8.5e-01   3.6e+00   7.3e-01   5.4e-01     4.9e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.4e-03   8.5e-01   3.5e+00   8.0e-01   6.9e-01     3.9e-01     9.7e-03    
Now optimizing over B

1     4.4e+00   9.6e-04   8.5e-01   3.5e+00   7.7e-01   6.2e-01     4.7e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.6e-03   8.5e-01   3.5e+00   8.6e-01   7.6e-01     4.0e-01     1.1e-02    
Now optimizing over B

1     4.4e+00   9.8e-04   8.5e-01   3.5e+00   7.8e-01   6.1e-01     4.8e-01    


1     4.4e+00   9.1e-04   8.6e-01   3.5e+00   7.1e-01   5.4e-01     4.7e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.4e-03   8.6e-01   3.5e+00   7.9e-01   6.9e-01     3.8e-01     9.7e-03    
Now optimizing over B

1     4.4e+00   9.2e-04   8.6e-01   3.5e+00   7.5e-01   6.0e-01     4.5e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.5e-03   8.6e-01   3.5e+00   8.4e-01   7.4e-01     4.0e-01     1.1e-02    
Now optimizing over B

1     4.4e+00   9.4e-04   8.6e-01   3.5e+00   7.6e-01   5.9e-01     4.7e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.7e-03   8.6e-01   3.5e+00   9.3e-01   8.0e-01     4.6e-01     1.1e-02    
Now optimizing over B

1     4.4e+00   1.1e-03   8.6e-01   3.5e+00   8.6e-01   6.8e-01     5.2e-01     3.8e-03    
Now optimizing over C

1     4.4e+00   1.8e-03   8.6e-01   3.5e+00   9.3e-01   8.1e-01     4.6e-01     1.1e-02    
Now optimizing over B

1     4.4e+00   1.1e-03   8.6e-01   3.5e+00   8.3e-01   6.4e-01     5.2e-01    


1     4.3e+00   1.1e-03   8.7e-01   3.5e+00   8.2e-01   6.2e-01     5.3e-01     3.8e-03    
Now optimizing over C

1     4.3e+00   1.6e-03   8.7e-01   3.5e+00   8.9e-01   7.6e-01     4.6e-01     9.7e-03    
Now optimizing over B

1     4.3e+00   1.1e-03   8.7e-01   3.5e+00   8.3e-01   6.6e-01     5.1e-01     3.8e-03    
Now optimizing over C

1     4.3e+00   1.8e-03   8.7e-01   3.5e+00   9.3e-01   7.9e-01     4.9e-01     1.1e-02    
Now optimizing over B

1     4.3e+00   1.2e-03   8.7e-01   3.5e+00   8.3e-01   6.4e-01     5.4e-01     3.8e-03    
Now optimizing over C

1     4.3e+00   1.6e-03   8.7e-01   3.5e+00   9.1e-01   7.7e-01     4.7e-01     9.7e-03    
Now optimizing over B

1     4.3e+00   1.2e-03   8.7e-01   3.5e+00   8.5e-01   6.8e-01     5.1e-01     3.8e-03    
Now optimizing over C

1     4.3e+00   1.8e-03   8.7e-01   3.5e+00   9.5e-01   8.1e-01     5.0e-01     1.1e-02    
Now optimizing over B

1     4.3e+00   1.2e-03   8.7e-01   3.5e+00   8.5e-01   6.5e-01     5.5e-01    


1     4.3e+00   1.7e-03   8.8e-01   3.4e+00   8.9e-01   7.5e-01     4.8e-01     1.1e-02    
Now optimizing over B

1     4.3e+00   1.1e-03   8.8e-01   3.4e+00   7.9e-01   5.8e-01     5.3e-01     3.8e-03    
Now optimizing over C

1     4.3e+00   1.6e-03   8.8e-01   3.4e+00   8.7e-01   7.3e-01     4.8e-01     9.7e-03    
Now optimizing over B

1     4.3e+00   1.1e-03   8.8e-01   3.4e+00   8.0e-01   6.1e-01     5.1e-01     3.8e-03    
Now optimizing over C

1     4.3e+00   1.5e-03   8.8e-01   3.4e+00   8.1e-01   6.9e-01     4.2e-01     9.7e-03    
Now optimizing over B

1     4.3e+00   9.9e-04   8.8e-01   3.4e+00   7.3e-01   5.5e-01     4.7e-01     3.8e-03    
Now optimizing over C

1     4.3e+00   1.4e-03   8.8e-01   3.4e+00   8.2e-01   7.0e-01     4.3e-01     9.7e-03    
Now optimizing over B

1     4.3e+00   9.9e-04   8.8e-01   3.4e+00   7.6e-01   6.0e-01     4.7e-01     3.8e-03    
Now optimizing over C

1     4.3e+00   1.6e-03   8.8e-01   3.4e+00   8.7e-01   7.4e-01     4.6e-01    

The norm of the gradient decreases by 6 orders of magnitude. The misfit (portion of the cost function that does not include the regularization terms) decreases by 11 orders of magnitude, for this particular randomly generated rank-r matrix A

In [20]:
def Newton(problem_handler,tolerance = 1e-4,max_iters =5000):
    problem = problem_handler
    grad_norm0 = problem.gradientnorm()
    grad_norm = grad_norm0
    iters = 0
    
    
    while grad_norm > tolerance*grad_norm0 and iters <= max_iters:
        misfit, regB, regC, cost = problem.cost()
        g = problem.gradient()
        P = problem.NewtonDirection()
        p_inner_g = np.sum(np.multiply(g[0],P[0])) + np.sum(np.multiply(g[1],P[1]))
        alpha = 0.1 #hardcoded stepsize that heuristically worked, line search did not work.
        problem.B += alpha*P[0]
        problem.C += alpha*P[1]
        grad_norm = problem.gradientnorm()

        if iters == 0:
            print ("\n{0:10} {1:15} {2:15}".format(
                  "Iteration",  "misfit", "||g||L2"))
        else:
            if ((iters%30) == 1):
                print ("\n{0:<10d} {1:<15e} {2:<15e}".format(
                  iters,  misfit, grad_norm))
        iters += 1
        alpha *=0.98
    print ("\n{0:<10d} {1:<15e} {2:<15e}".format(
                  iters,  misfit, grad_norm))
       
    return problem.B,problem.C 

Newton with fixed stepsize alpha = 0.01, because Backtracking didn't work well

In [21]:
#test n = 10, r = 5, any larger, too long to form the hessian
import numpy as np
import numpy.linalg as la

n = 10
r = 5
B =  np.random.rand(n,r)
C =  np.random.rand(r,n)
A = np.matmul(B,C) #generate a rank-r matrix A. Otherwise, this problem is pointless. 
#Needs to be rank-r for this problem to be meaningful.

mean = 0.0
sigma = 5.0
B = np.random.normal(loc=mean,scale=sigma,size=(n,r))
C = np.random.normal(loc=mean,scale=sigma,size=(r,n))

#Not too sparse, but whatever, TA said it doesnt really matter too much how sparse.
# G = np.random.randint(2, size=(n,n))
G = np.eye(n,n) #for this problem, 
#since I couldnt afford larger matrices, because of the cost of forming the hessian,
#G must be identity or close, otherwise, we wont be able to recover A with a B and C.
#too sparse information.. 
#That is the justification for using G = Identity in this case.

problem = Problem2(A,B,C,G)


B,C = Newton(problem)
print("Final Frobenius distance:"+str(np.linalg.norm(A-np.matmul(B,C))))


Iteration  misfit          ||g||L2        

1          4.540039e+04    6.807164e+03   

31         9.776768e+02    3.426181e+02   

61         2.641701e+01    1.626583e+01   

91         1.017802e+01    8.105280e-01   

94         1.015199e+01    6.877270e-01   
Final Frobenius distance:236.933421554


I wasnt able to get more than 4 orders of magnitude decrease in the gradient norm with regular Newton

In [29]:
def TrustRegionNewton(problem_handler,tolerance = 1e-4,delta_hat = 1.,delta_0_rel = 1.0, eta = 0.25):
    problem = problem_handler
    grad_norm0 = problem.gradientnorm()
    grad_norm = grad_norm0
    
    iters = 0
    
    delta_0 = delta_0_rel*delta_hat
    delta = delta_0
    
    while grad_norm > tolerance*grad_norm0:
        misfit, regB, regC, cost_0 = problem.cost()
        P = problem.NewtonDirection()
        normOfDirection = problem.FrobeniusNorm(P) 
        if normOfDirection > delta:
            P = problem.RescaleDirection(P,delta/normOfDirection)
            normOfDirection = delta
            
        predictedRed = problem.PredictedReduction(P)
        B_new = problem.B +P[0]
        C_new = problem.C +P[1]
        misfit, regB, regC,cost_p = problem.cost(B=B_new,C=C_new) 
        actualRed = cost_0 - cost_p
        rho = actualRed/predictedRed
        if rho < 0.25:
            delta *= 0.25
        else:
            if (rho > 0.75) and (normOfDirection == delta):
                delta = min(2*delta,delta_hat)
        if rho > eta:
            problem.B += P[0]
            problem.C += P[1]
            
        grad_norm = problem.gradientnorm()    
        misfit, regB, regC, cost = problem.cost()
        if iters == 0:
            print("\n{0:4} {1:9} {2:9} {3:9} {4:9} {5:9}".format(
                      "It",  "cost", "misfit","reg","||grad||","rho" ))
        else:
            if ((iters%10) == 1):
                print("\n{0:<5d} {1:<9.1e} {2:<9.1e} {3:<9.1e} {4:<9.1e} {5:<9.1e}".format(
              iters,  cost, misfit,regB+regC,grad_norm, rho))
        iters += 1
    return problem.B,problem.C 

Trust Region Newton decreases 4 orders of magntitude in gradient norm for this particular problem in very few iterations

In [30]:
#test n = 15, r = 7, any larger, too long to form the hessian
import numpy as np
import numpy.linalg as la

n = 15
r = 7
B =  np.random.rand(n,r)
C =  np.random.rand(r,n)
A = np.matmul(B,C) #generate a rank-r matrix A. Otherwise, this problem is pointless. 
#Needs to be rank-r for this problem to be meaningful.

mean = 0.0
sigma = 5.0
B = np.random.normal(loc=mean,scale=sigma,size=(n,r))
C = np.random.normal(loc=mean,scale=sigma,size=(r,n))

#Not too sparse, but whatever, TA said it doesnt really matter too much how sparse.
# G = np.random.randint(2, size=(n,n))
G = np.eye(n,n) #for this problem, 
#since I couldnt afford larger matrices, because of the cost of forming the hessian,
#G must be identity or close, otherwise, we wont be able to recover A with a B and C.
#too sparse information.. 
#That is the justification for using G = Identity in this case.

problem = Problem2(A,B,C,G)

B,C = TrustRegionNewton(problem)
print("Final Frobenius distance:"+str(np.linalg.norm(A-np.matmul(B,C))))



It   cost      misfit    reg       ||grad||  rho      

1     6.3e+04   6.3e+04   2.8e+01   1.1e+04   9.9e-01  

11    2.7e+04   2.7e+04   2.7e+01   5.7e+03   9.9e-01  

21    5.4e+03   5.4e+03   2.5e+01   1.5e+03   9.9e-01  

31    1.9e+02   1.7e+02   2.4e+01   1.1e+02   9.8e-01  
Final Frobenius distance:905.603320576


I actually found Newton with a heuristically chosen constant stepsize to work better than Trust Region Newton. 

Here we try a slightly larger problem (n x r) = (15,7), but anything larger was infeasible to solve in a reasonable time. The reason is that forming the hessian is impractical (2n x r) x (2n x r). Thus, it is noted that ultimately, one would want to use an iterative method for the linear solve. Something like CG with early stopping probably would work. Thus, we would not need to form the hessian, but only need the hessian action, which would require only forming the (2n x r) hessian action on a matrix. rather than the full hessian tensor (and inverting the hessian is insanely expensive).